[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/philschmid/gemini-2.5-ai-engineering-workshop/blob/main/notebooks/04-model-context-protocol-mcp.ipynb)

# Part 4: Model Context Protocol (MCP)

The Model Context Protocol (MCP) is an open standard for connecting AI assistants to external data sources and tools. It enables seamless integration between LLMs and various services, databases, and APIs through a standardized protocol.

In [1]:
%pip install mcp

  Using cached mcp-1.9.2-py3-none-any.whl.metadata (28 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached sse_starlette-2.3.6-py3-none-any.whl.metadata (10 kB)
  Using cached starlette-0.47.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached uvicorn-0.34.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached mcp-1.9.2-py3-none-any.whl (131 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.9.1-py3-none-any.whl (44 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Using cached python_multipart-0.0.20-py3-none-any.whl (24 kB)
Using cached sse_starlette-2.3.6-py3-none-any.whl (10 kB)
Using cached starlette-0.47.0-py3-none-any.whl (72 kB)
Using cached uvicorn-0.34.3-py3-none-any.whl (62 kB)
   ━━

In [1]:
from google import genai
from google.genai import types
import sys
import os
import asyncio
from datetime import datetime
from mcp import ClientSession, StdioServerParameters
from mcp.client.streamable_http import streamablehttp_client
from mcp.client.stdio import stdio_client

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
else:
    GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY',None)

# Create client with api key
MODEL_ID = "gemini-2.5-flash-preview-05-20"
client = genai.Client(api_key=GEMINI_API_KEY)

## What is MCP?

Model Context Protocol (MCP) is a revolutionary approach to extending AI capabilities. Unlike traditional function calling where you define functions locally in your code, MCP allows AI models to connect to remote servers that provide tools and resources.


- **🔌 Plug-and-Play Integration**: Connect to any MCP-compatible service instantly
- **🌐 Remote Capabilities**: Access tools and data from anywhere on the internet
- **🔄 Standardized Protocol**: One protocol works with all MCP servers
- **🔒 Centralized Security**: Control access and permissions at the server level
- **📈 Scalability**: Share resources across multiple AI applications
- **🛠️ Rich Ecosystem**: Growing library of MCP servers for various use case

## 1. Working with Stdio MCP Servers

Stdio (Standard Input/Output) servers run as local processes and communicate through pipes. This is perfect for:
- Development and testing
- Local tools and utilities
- Lightweight integrations


## 1. Working with MCP Servers

Let's use the DeepWiki MCP server, which provides access to Wikipedia data and search capabilities:

In [3]:
# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="npx",  # Executable
    args=["-y", "@philschmid/weather-mcp"],  # MCP Server
    env=None,  # Optional environment variables
)

async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Prompt to get the weather for the current day in London.
            prompt = f"What is the weather in London in {datetime.now().strftime('%Y-%m-%d')}?"
            # Initialize the connection between client and server
            await session.initialize()
            # Send request to the model with MCP function declarations
            response = await client.aio.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt,
                config=genai.types.GenerateContentConfig(
                    temperature=0,
                    tools=[session],  # uses the session, will automatically call the tool
                    # Uncomment if you **don't** want the sdk to automatically call the tool
                    # automatic_function_calling=genai.types.AutomaticFunctionCallingConfig(
                    #     disable=True
                    # ),
                ),
            )
            print(response.text)

await run()

  + Exception Group Traceback (most recent call last):
  |   File "/Users/ferveloz/data/formaciones/gemini-2.5-ai-engineering-workshop/gemini_2.5_venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3670, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/h6/7xcgpb4s0j5d9cjp5x2_d27m0000gn/T/ipykernel_51299/52228023.py", line 30, in <module>
  |     await run()
  |   File "/var/folders/h6/7xcgpb4s0j5d9cjp5x2_d27m0000gn/T/ipykernel_51299/52228023.py", line 9, in run
  |     async with stdio_client(server_params) as (read, write):
  |                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/Users/ferveloz/data/formaciones/gemini-2.5-ai-engineering-workshop/gemini_2.5_venv/lib/python3.12/site-packages/mcp/client/stdio/__init__.py", line 179, in stdio_client
  |     anyi

## !! Exercise: Build Your Own MCP CLI Agent !!

Create an interactive command-line interface (CLI) chat agent that connects to the DeepWiki MCP server (a remote server providing access to Wikipedia-like data). The agent should allow users to ask questions about GitHub repositories, and it will use the DeepWiki server to find answers.

Task:
- Use `mcp.client.streamable_http.streamablehttp_client` to establish a connection to the remote URL (https://mcp.deepwiki.com/mcp). 
- Inside the `async with streamablehttp_client(...)` block, create an `mcp.ClientSession`.
- Initialize the session using `await session.initialize()`.
- Create a `genai.types.GenerateContentConfig` with `temperature=0` and pass the `session` object in the `tools` list. This configures the chat to use the MCP server.
- Create an asynchronous chat session using `client.aio.chats.create()`, passing the `MODEL_ID` (e.g., "gemini-2.5-flash-preview-05-20") and the `config` you created.
- Implement an interactive loop to chat with the model using `input()` to get the user's input.

In [3]:
remote_url = "https://mcp.deepwiki.com/mcp"

async def run():
    # - Use `mcp.client.streamable_http.streamablehttp_client` to establish a connection to the remote URL (https://mcp.deepwiki.com/mcp). 
    async with streamablehttp_client(remote_url) as (read, write, _):
        # - Inside the `async with streamablehttp_client(...)` block, create an `mcp.ClientSession`.
        async with ClientSession(read, write) as session:
            # - Initialize the session using `await session.initialize()`.
            await session.initialize()
            # - Create a `genai.types.GenerateContentConfig` with `temperature=0` and pass the `session` object in the `tools` list. This configures the chat to use the MCP server.
            config = genai.types.GenerateContentConfig(
                    temperature=0,
                    tools=[session],  # uses the session, will automatically call the tool
                    # Uncomment if you **don't** want the sdk to automatically call the tool
                    # automatic_function_calling=genai.types.AutomaticFunctionCallingConfig(
                    #     disable=True
                    # ),
                )
            # Send request to the model with MCP function declarations
            print("Agent is ready. Type 'exit' to quit.")
            # - Create an asynchronous chat session using `client.aio.chats.create()`, passing the `MODEL_ID` (e.g., "gemini-2.5-flash-preview-05-20") and the `config` you created.
            chat =  client.aio.chats.create(
                model=MODEL_ID,
                config=config,
            )
            # - Implement an interactive loop to chat with the model using `input()` to get the user's input.
            while True:
                user_input = input("You: ")
                if user_input.lower() == "exit":
                    print("Exiting chat.")
                    break
                print(user_input)
                # Append user message to history
                response = await chat.send_message(user_input)
                if len(response.automatic_function_calling_history) > 0:
                    if (
                        response.automatic_function_calling_history[0].parts[0].text
                        == user_input
                    ):
                        response.automatic_function_calling_history.pop(0)
                    for call in response.automatic_function_calling_history:
                        if call.parts[0].function_call:
                            print(f"Function call: {call.parts[0].function_call}")
                        elif call.parts[0].function_response:
                            print(
                                f"Function response: {call.parts[0].function_response.response['result'].content[0].text}"
                            )
                print(f"Assistant: {response.text}")

await run()

Agent is ready. Type 'exit' to quit.
hi! Can you talk a bit about huggingface/transformers repository?
Function call: id=None args={'repoName': 'huggingface/transformers'} name='read_wiki_contents'
Function response: # Page: Overview

# Overview

<details>
<summary>Relevant source files</summary>

The following files were used as context for generating this wiki page:

- [README.md](README.md)
- [docs/source/en/_toctree.yml](docs/source/en/_toctree.yml)
- [docs/source/en/index.md](docs/source/en/index.md)
- [docs/source/en/perf_infer_gpu_one.md](docs/source/en/perf_infer_gpu_one.md)
- [docs/source/ko/_toctree.yml](docs/source/ko/_toctree.yml)
- [docs/source/ko/model_doc/qwen2_vl.md](docs/source/ko/model_doc/qwen2_vl.md)
- [src/transformers/__init__.py](src/transformers/__init__.py)
- [src/transformers/models/__init__.py](src/transformers/models/__init__.py)
- [src/transformers/models/auto/configuration_auto.py](src/transformers/models/auto/configuration_auto.py)
- [src/transformers/mod

## Recap & Next Steps

**What You've Learned:**
- Understanding the Model Context Protocol (MCP) and its advantages over traditional function calling
- Connecting to remote MCP servers using both stdio and HTTP protocols
- Building interactive chat agents that leverage MCP capabilities

**Key Takeaways:**
- MCP enables plug-and-play integration with external services and data sources
- Remote capabilities provide access to tools and data from anywhere on the internet
- Standardized protocols ensure compatibility across different AI applications
- Centralized security and permissions improve enterprise deployment scenarios
- The MCP ecosystem is rapidly growing with servers for various use cases

🎉 **Congratulations!** You've completed the Gemini 2.5 AI Engineering Workshop

**More Resources:**
- [MCP with Gemini Documentation](https://ai.google.dev/gemini-api/docs/function-calling?example=weather#model_context_protocol_mcp)
- [Function Calling Documentation](https://ai.google.dev/gemini-api/docs/function-calling?lang=python)
- [MCP Official Specification](https://spec.modelcontextprotocol.io/)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP Server Directory](https://github.com/modelcontextprotocol/servers)